# Preprocesamiento

    - Borro duplicados

    - Seleciono idpostulantes y idavisos

    - Postulantes Educacion

    - Postulantes Genero y Edad

    - Postulaciones
    
    - Vistas

    - Avisos Online

    - Avisos Detalle
    

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

In [2]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Load CSV

In [3]:
test_final = pd.read_csv('data/test_final_100k.csv')
test_final = test_final.drop(columns='id')

In [4]:
# idpostulantes
postulantes_educacion = pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.reset_index(drop=True)

In [5]:
# idpostulantes
postulantes_genero_y_edad = pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [6]:
# idpostulantes idaviso
vistas = pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index(drop=True)
vistas = vistas.rename(columns={'idAviso':'idaviso'})

In [7]:
# idpostulantes idaviso
postulaciones = pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index(drop=True)

In [8]:
# idaviso
avisos_online = pd.read_csv('data/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv'))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')
avisos_online = avisos_online.reset_index(drop=True)

In [9]:
# idaviso
avisos_detalle = pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index(drop=True)

# Borro duplicados

    - supongo que la ultima aparicion es el registro más reciente

In [10]:
postulantes_educacion = postulantes_educacion.drop_duplicates(keep='last')
postulantes_educacion = postulantes_educacion.reset_index(drop=True)

In [11]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante',keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [12]:
vistas = vistas.drop_duplicates(keep='last')
vistas = vistas.reset_index(drop=True)

In [13]:
postulaciones = postulaciones.drop_duplicates()
postulaciones = postulaciones.reset_index(drop=True)
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='last')

In [14]:
avisos_online = avisos_online.drop_duplicates(keep='last')
avisos_online = avisos_online.reset_index(drop=True)

In [15]:
avisos_detalle = avisos_detalle.drop_duplicates('idaviso',keep='last')
avisos_detalle = avisos_detalle.reset_index(drop=True)

# Selecciono idpostulantes y idavisos

Correr solo el deseado

    - Gente solo_test
    - Avisos solo_test
    - Gente en comun entre postulantes_genero_y_edad + test
    - Avisos solo aviso_detalle

# gente_solo_test
gente = test_final['idpostulante'].drop_duplicates()
postulantes_educacion = postulantes_educacion[postulantes_educacion['idpostulante'].isin(gente)]
postulantes_genero_y_edad = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].isin(gente)]
vistas = vistas[vistas['idpostulante'].isin(gente)]
postulaciones = postulaciones[postulaciones['idpostulante'].isin(gente)]

# avisos_solo_test
avisos = test_final['idaviso'].drop_duplicates()
vistas = vistas[vistas['idaviso'].isin(avisos)]
postulaciones = postulaciones[postulaciones['idaviso'].isin(avisos)]
avisos_online = avisos_online[avisos_online['idaviso'].isin(avisos)]
avisos_detalle = avisos_detalle[avisos_detalle['idaviso'].isin(avisos)]

In [16]:
# gente pge,pe y appendeo gente test
gente = postulantes_genero_y_edad.merge(postulantes_educacion,how='outer',on='idpostulante',indicator=True)
gente = gente[gente['_merge'] == 'both']
gente = gente.drop(columns='_merge')
gente = gente['idpostulante'].drop_duplicates()
gente = gente.append(test_final['idpostulante'])
gente = gente.drop_duplicates()
postulantes_educacion = postulantes_educacion[postulantes_educacion['idpostulante'].isin(gente)]
postulantes_genero_y_edad = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].isin(gente)]
vistas = vistas[vistas['idpostulante'].isin(gente)]
postulaciones = postulaciones[postulaciones['idpostulante'].isin(gente)]

In [17]:
# avisos solo detalles
avisos = avisos_detalle['idaviso'].drop_duplicates()
vistas = vistas[vistas['idaviso'].isin(avisos)]
postulaciones = postulaciones[postulaciones['idaviso'].isin(avisos)]
avisos_online = avisos_online[avisos_online['idaviso'].isin(avisos)]
avisos_detalle = avisos_detalle[avisos_detalle['idaviso'].isin(avisos)]

# Tasas de postulacion segun idaviso y idpostulante

In [18]:
# paso a datetime
postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones['fechapostulacion'])
postulaciones['fechapostulacion'] = postulaciones['fechapostulacion'].dt.date
vistas['timestamp'] = pd.to_datetime(vistas['timestamp'])
vistas['timestamp'] = vistas['timestamp'].dt.date

In [19]:
# tasa de postulaciones segun aviso
postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(vistas['timestamp'].drop_duplicates())]
vistas_validas = vistas[vistas['timestamp'].isin(postulaciones_validas['fechapostulacion'].drop_duplicates())]

postulaciones_validas = postulaciones_validas.groupby('idaviso').agg({'idpostulante':'count'})
postulaciones_validas = postulaciones_validas.reset_index()
vistas_validas = vistas_validas.groupby('idaviso').agg({'idpostulante':'count'})
vistas_validas = vistas_validas.reset_index()

tasa_idaviso = postulaciones_validas.merge(vistas_validas,on='idaviso')
tasa_idaviso['tasa_idaviso'] = tasa_idaviso['idpostulante_x'] / tasa_idaviso['idpostulante_y']
tasa_idaviso = tasa_idaviso.drop(columns=['idpostulante_x','idpostulante_y'])

In [20]:
# tasa de postulaciones segun postulante
postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(vistas['timestamp'].drop_duplicates())]
vistas_validas = vistas[vistas['timestamp'].isin(postulaciones_validas['fechapostulacion'].drop_duplicates())]

postulaciones_validas = postulaciones_validas.groupby('idpostulante').agg({'idaviso':'count'})
postulaciones_validas = postulaciones_validas.reset_index()
vistas_validas = vistas_validas.groupby('idpostulante').agg({'idaviso':'count'})
vistas_validas = vistas_validas.reset_index()

tasa_idpostulante = postulaciones_validas.merge(vistas_validas,on='idpostulante')
tasa_idpostulante['tasa_idpostulante'] = tasa_idpostulante['idaviso_x'] / tasa_idpostulante['idaviso_y']
tasa_idpostulante = tasa_idpostulante.drop(columns=['idaviso_x','idaviso_y'])

# Postulantes Educacion

    - Pondero el estado de la educacion y separo en columnas segun el nivel

In [21]:
postulantes_educacion.head()

,idpostulante,nombre,estado
0,8BkL,Universitario,En Curso
1,1d2B,Universitario,En Curso
2,6M3jr,Universitario,En Curso
3,Z85B5,Universitario,En Curso
4,1QLLO,Universitario,En Curso


In [22]:
postulantes_educacion = postulantes_educacion.drop_duplicates(keep='last')
postulantes_educacion = postulantes_educacion.reset_index(drop=True)

In [23]:
dic = {'Graduado': 3,'En Curso': 2,'Abandonado': 1}
postulantes_educacion['estado'] = postulantes_educacion['estado'].map(dic)
postulantes_educacion = postulantes_educacion.groupby(['idpostulante','nombre']).max().unstack(fill_value=0)
postulantes_educacion.columns = postulantes_educacion.columns.droplevel()
postulantes_educacion = postulantes_educacion.reset_index()
postulantes_educacion = postulantes_educacion.rename_axis('',axis='columns')

In [24]:
postulantes_educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,3,0,3,3,1
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,3,1
4,0zB026d,0,0,0,0,3,3,0


In [25]:
postulantes_educacion.to_csv('data_preprocesada/fiuba_1_postulantes_educacion.csv')
del postulantes_educacion

# Postulantes Genero y Edad

    - Salvo las edades con la mediana
    - Salvo el genero con NO_DECLARA
    - Separo en columnas el genero

In [26]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo
0,eo2p,1981-02-16,MASC
1,1d2B,1976-02-28,MASC
2,EBO0,1973-09-11,FEM
3,a6MKW,1974-06-07,MASC
4,6MWd4,1974-12-19,MASC


In [27]:
postulantes_genero_y_edad.shape

(452626, 3)

In [28]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante',keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [29]:
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isna(),'edad'] = postulantes_genero_y_edad['edad'].median()
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int16')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='fechanacimiento')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [30]:
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['sexo'] == '0.0','sexo'] = 'NO_DECLARA'
postulantes_genero_y_edad = oneHotEncoding(postulantes_genero_y_edad,'sexo')

In [31]:
# edad en rangos
#postulantes_genero_y_edad['edad'] = pd.cut(postulantes_genero_y_edad['edad'], range(10, 106, 5))
#postulantes_genero_y_edad = oneHotEncoding(postulantes_genero_y_edad,'edad')

In [32]:
postulantes_genero_y_edad = postulantes_genero_y_edad.merge(tasa_idpostulante,how='left',on='idpostulante')

In [33]:
postulantes_genero_y_edad = postulantes_genero_y_edad.fillna(0.5)

In [34]:
postulantes_genero_y_edad.head()

,idpostulante,edad,FEM,MASC,NO_DECLARA,tasa_idpostulante
0,eo2p,37,0,1,0,0.5
1,1d2B,42,0,1,0,0.5
2,EBO0,45,1,0,0,0.5
3,a6MKW,44,0,1,0,0.5
4,6MWd4,44,0,1,0,0.5


In [35]:
postulantes_genero_y_edad.to_csv('data_preprocesada/fiuba_2_postulantes_genero_y_edad.csv')
del postulantes_genero_y_edad

# Postulaciones

In [36]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15
1,1111920714,NM5M,2018-02-06
2,1112346945,NM5M,2018-02-22
3,1112345547,NM5M,2018-02-22
4,1112237522,5awk,2018-01-25


In [37]:
postulaciones = postulaciones.drop(columns='fechapostulacion')

In [38]:
postulaciones = postulaciones.drop_duplicates()
postulaciones = postulaciones.reset_index(drop=True)

In [39]:
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [40]:
postulaciones.to_csv('data_preprocesada/fiuba_4_postulaciones.csv')
del postulaciones

# Vistas

    - Genero una marca para las tuplas visitadas

In [41]:
vistas.head()

,idaviso,timestamp,idpostulante
0,1111780242,2018-02-23,YjVJQ6Z
1,1112263876,2018-02-23,BmVpYoR
2,1112327963,2018-02-23,wVkBzZd
3,1112318643,2018-02-23,OqmP9pv
4,1111903673,2018-02-23,DrpbXDP


In [42]:
vistas = vistas.drop_duplicates(keep='last')
vistas = vistas.reset_index(drop=True)

In [43]:
vistas = vistas.drop(columns='timestamp')

In [44]:
vistas['visto'] = 1
vistas = vistas.groupby(['idaviso','idpostulante']).max()
vistas = vistas.reset_index()

In [45]:
vistas.head()

,idaviso,idpostulante,visto
0,820850,DrXDRVP,1
1,1112360,dY98A2D,1
2,1141070,1QPJWqB,1
3,1141070,5maBxez,1
4,1141070,QNEdKBW,1


In [46]:
vistas.to_csv('data_preprocesada/fiuba_3_vistas.csv')
del vistas

# Avisos Online

    - Genero una marca para los avisos online

In [47]:
avisos_online.head()

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480


In [48]:
avisos_online = avisos_online.drop_duplicates(keep='last')
avisos_online = avisos_online.reset_index(drop=True)

In [49]:
avisos_online['online'] = 1

In [50]:
avisos_online.head()

,idaviso,online
0,1112355872,1
1,1112335374,1
2,1112374842,1
3,1111984070,1
4,1111822480,1


In [51]:
avisos_online.to_csv('data_preprocesada/fiuba_5_avisos_online.csv')
del avisos_online

# Avisos Detalle

    - Agrupamos los nombres de area en subconjuntos más grandes
    - Agrego tasa del detalle

In [52]:
avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL
1,9240880,1,"Productores Asesores Independiente, para venta...",Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga
2,1110185164,1,Administrativo de Recepción,<p>Centro Médico Accord se encuentra en la bús...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Administración,Unión Personal
3,1110513885,1,Distribuidor domiciliario con moto (SAN MIGUEL),<p>Importante correo privado ubicado en SAN MI...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Distribución,Asoko Tempo SA
4,1111034024,1,Vendedores para venta de medicina pre paga y o...,<p> </p><p><strong>Comercializadora incorpora ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,JELS SRL


In [53]:
avisos_detalle = avisos_detalle.drop(columns=['idpais','titulo','descripcion','nombre_zona','ciudad','mapacalle','denominacion_empresa'])

In [54]:
avisos_detalle = avisos_detalle.drop_duplicates('idaviso',keep='last')
avisos_detalle = avisos_detalle.reset_index(drop=True)

In [55]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [56]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

In [57]:
avisos_detalle = oneHotEncoding(avisos_detalle,'nombre_area')

avisos_detalle = oneHotEncoding(avisos_detalle,'tipo_de_trabajo')

avisos_detalle = oneHotEncoding(avisos_detalle,'nivel_laboral')

In [58]:
avisos_detalle = avisos_detalle.drop(columns=['tipo_de_trabajo','nivel_laboral'])

In [59]:
# agrego tasa
avisos_detalle = avisos_detalle.merge(tasa_idaviso,how='left',on='idaviso')
avisos_detalle = avisos_detalle.fillna(0.5)

In [60]:
avisos_detalle.head()

,idaviso,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,Diseño,Diseño Otros,Diseño y Urbanismo,Educación,...,Salud,Secretaria,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero,tasa_idaviso
0,1000872556,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
1,9240880,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.5
2,1110185164,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
3,1110513885,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
4,1111034024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.5


In [61]:
avisos_detalle.to_csv('data_preprocesada/fiuba_6_avisos_detalle.csv')
del avisos_detalle